#### 0. Importando paquetes

In [1]:
import zipfile
import os
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector

import glob

#### 1. Cargamos los CSV que genera Dariela de las cargas en los loadboards

Son 4 archivos que vienen de DAT, 123 Loadboard, DOFT y TruckerPath.

In [46]:
df=pd.read_parquet("loadboards_sample_day (2).parquet")

In [ ]:
with zipfile.ZipFile('drive-download-20231107T133047Z-001.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
archivos_csv = glob.glob('*.csv')
df = []

#### 2. Limpiando la base de datos
En este paso, eliminamos las cargas duplicadas, modificamos el tipo de datos en algunas columnas y filtramos a cargas disponibles hoy.

In [47]:
#concatenando bases de datos
#for archivo in archivos_csv:
 #   dfx = pd.read_csv(archivo)
  #  df.append(dfx)
#df = pd.concat(df, ignore_index=True)

#cambiando tipo de variable de algunas columnas
df['Removed'] = pd.to_datetime(df['Removed'])
df['Posted'] = pd.to_datetime(df['Posted'])
df['PickUp'] = pd.to_datetime(df['PickUp'])
df['ID']=df['ID'].astype(str)

#Dejando solo las cargas que siguen disponibles
ayer = datetime.now() - timedelta(days=1)
df = df.loc[df['Removed']>= ayer]

#Eliminando los valores duplicados
df=df.drop_duplicates(subset='ID', keep='first')

#Dejando solo las cargas que aplican al tipo de camiones que estamos manejando
trucksok=['Any','Flatbed','Reefer','Truck&Trailer','Van']
df=df[df['Equip'].apply(lambda x: any(word in x for word in trucksok))]

#Dejando solo cargas FTL
df=df[df['Type']=='TL']

In [48]:
df['Source'].unique()

array(['DAT', '123LoadBoard', 'TruckerPath'], dtype=object)

In [41]:
columnsoutput=['CityOrigin', 'CountyOrigin', 'HubOrigin','PickUp', 'CityDestination',  'CountyDestination', 
               'HubDestination',#'DropOff', 
               'Distance','RatePerMile','Broker_Shipper',
               'Broker_Shipper_Email', 'Broker_Shipper_Phone','Weight','Size',   
               #'PickUp_Stops', 
               #'DropOff_Stops', 
               'Specifications', 
               #'Commodity',
               'Equip','ID']

df=df[columnsoutput]

#### 3. Exportamos la base de datos
Finalmente, generamos el archivo .PARQUET que se utiliza en la aplicación de búsqueda de carga

In [42]:
df.to_parquet("dfloads.parquet")

In [ ]:
#hay que conectar esto con el drive así como lo tiene Dariela
#automatizar más la entrada de las nuevas cargas
#Los sábados sería el único día que no descargaríamos info de los loadboards

In [210]:
#tipos de camiones
trucktypes=['', 'Any', 'Auto', 'AutoCarrier', 'BoatHauler', 'BoxTruck',
 'CargoVan', 'Conestoga', 'Container', 'DoubleDrop', 'DumpTruck', 'Flatbed', 'HeavyHaulers', 'HopperBottom', 'Hotshot', 'Landoll', 'Lowboy', 'MovingVan',
 'PickupTruck', 'PowerOnly', 'RGN', 'Reefer', 'RemovableGooseneck', 'Sprinter', 'StepDeck', 'Tanker','Truck&Trailer', 'Van']